In [ ]:
# replace this with a Spark history log of your own or parameterize with Papermill!

metrics_file = "metrics/application_1601392010735_0030"

output_file = "output.db"

In [ ]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [ ]:
import pyspark
import pyspark.sql.functions as F
import json

spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [ ]:
metrics = spark.read.json(metrics_file)

In [ ]:
app_id, app_name = metrics.select("App ID", "App Name").dropna().collect()[0]

def with_appmeta(df):
    return df.withColumn("Application ID", F.lit(app_id)).withColumn("Application Name", F.lit(app_name))

In [ ]:
def collect_and_dictify(df):
    return [json.loads(row[0]) for row in df.selectExpr("to_json(*)").collect()]

def executor_info(df):
    info = df.select("Executor Info").dropna()
    return collect_and_dictify(info)

def plan_dicts(df):
    return collect_and_dictify(df.select("sparkPlanInfo").dropna())


In [ ]:
from collections import namedtuple

MetricNode = namedtuple("MetricNode", "plan_node accumulatorId metricType name")
PlanInfoNode = namedtuple("PlanInfoNode", "plan_node parent nodeName simpleString")

def nextid():
    i = 0
    while True:
        yield i
        i = i + 1
    
node_ctr = nextid()

def plan_dicts(df):
    return collect_and_dictify(df.select("sparkPlanInfo").dropna())

def flatplan(dicts, parent=-1, plan_nodes=None, metric_nodes=None):
    if plan_nodes is None:
        plan_nodes = list()
        
    if metric_nodes is None:
        metric_nodes = list()
    
    for pd in dicts:
        pid = next(node_ctr)
        for m in pd['metrics']:
            metric_nodes.append(MetricNode(pid, m['accumulatorId'], m['metricType'], m['name']))
        
        plan_nodes.append(PlanInfoNode(pid, parent, pd['nodeName'], pd['simpleString']))
        
        flatplan(pd['children'], pid, plan_nodes, metric_nodes)
    
    return(plan_nodes, metric_nodes)

def plan_dfs(df):
    pn, mn = flatplan(plan_dicts(metrics))
    
    pndf = with_appmeta(spark.createDataFrame(data=pn))
    mndf = with_appmeta(spark.createDataFrame(data=mn))
    
    return (pndf, mndf)

In [ ]:
plan_nodes, accumulable_nodes = plan_dfs(metrics)
pn, mn = flatplan(plan_dicts(metrics))

In [ ]:
def tasks_to_stages(df):
    return df.where(F.col('Event') == 'SparkListenerTaskStart').select(F.col("Task Info.Task ID").alias('Task ID'), 'Stage ID')

In [ ]:
def accumulables(df, noun='Task', extra_cols=[]):
    mcol = '%s Info' % noun
    idcol = '%s ID' % noun
    
    acc_cols = [F.col('Accumulable.%s' % s).alias('Metric %s' % s) for s in ['ID', 'Name', 'Value']]
    obs = df.select(mcol, *extra_cols).select('%s.*' % mcol, *extra_cols)
    cols = [F.col(elt) for elt in sorted(set(obs.columns) - set([idcol, 'Accumulables']))]
    
    return obs.select(
        idcol, 
        F.explode('Accumulables').alias('Accumulable'), 
        *(cols)
    ).select(
        idcol, 
        *(cols + acc_cols)
    ).withColumnRenamed("Metric ID", "accumulatorId").withColumn("Metric Value", F.col("Metric Value").cast("float"))

def explicit_task_metrics(df, noun="Task"):
    def flatten_once(df):
        import pyspark.sql.types as T

        cols = []
    
        for column in df.schema:
            if isinstance(column.dataType, T.StructType):
                for subfield in column.dataType:
                    cols.append(F.col("%s.%s" % (column.name, subfield.name)).alias("%s %s" % (column.name, subfield.name)))
            elif isinstance(column.dataType, T.ArrayType):
                pass
            else:
                cols.append(F.col(column.name))

        return df.select(*cols)
    
    mcol = '%s Info' % noun
    idcol = '%s ID' % noun
    
    obs = df.select('%s.*' % mcol, "Stage ID", F.col("Task Metrics.*"), F.col("Task Executor Metrics.*"))
    cols = [F.col(elt) for elt in obs.columns if elt not in set([idcol, 'Accumulables'])]

    flat_df = flatten_once(obs.select(idcol, *cols))
    
    common_split = flat_df.columns.index("Stage ID") + 1
    
    common_columns = flat_df.columns[:common_split]
    metric_columns = flat_df.columns[common_split:]
    
    result = flat_df.where(
        F.lit(False)
    ).select(
        *(common_columns + 
          [
              F.lit(None).alias("accumulatorID"), 
              F.lit(None).alias("Metric Name"), 
              F.lit(None).alias("Metric Value")
          ]
         )
    )
    
    for col in metric_columns:
        result = result.union(
            flat_df.select(
                *(
                    common_columns + 
                    [
                        F.lit(None).alias("accumulatorID"), 
                        F.lit(col).alias("Metric Name"), 
                        F.col(col).alias("Metric Value")
                    ]
                )
            )
        )
    
    return result

def tidy_metrics(df, noun='Task', event=None, interesting_metrics=None, extra_cols=[]):
    mcol = '%s Info' % noun
    idcol = '%s ID' % noun
    
    if event is not None:
        event_selector = (F.col('Event') == event)
    else:
        event_selector = F.lit(True)
    
    filtered = df.where(event_selector)
    
    return with_appmeta(accumulables(filtered, noun, extra_cols).unionByName(explicit_task_metrics(filtered)))

def tidy_tasks(df, event='SparkListenerTaskEnd', interesting_metrics=None):
    return tidy_metrics(df, 'Task', event=event, interesting_metrics=(interesting_metrics or F.lit(True)), extra_cols=["Stage ID"])

def tidy_stages(df, event='SparkListenerStageCompleted', interesting_metrics=None):
    return tidy_metrics(df, 'Stage', event=event, interesting_metrics=F.lit(True))



In [ ]:
def split_metrics(df):
    metric_columns = set(["Metric Name", "Metric Value", "accumulatorId", "kind", "unit"])
    common_columns = set(["Application ID", "Application Name", "Task ID", "Stage ID"])
    
    metrics = df.select(*[col for col in df.columns if col in metric_columns or col in common_columns])
    task_meta = df.drop(*metric_columns).distinct()
    
    return (metrics, task_meta)

In [ ]:
MetricMeta = namedtuple('MetricMeta', 'MetricName kind unit')

metric_metas = [
    MetricMeta('GPU decode time', 'time', 'ms'),
    MetricMeta('GPU time', 'time', 'ms'),
    MetricMeta('avg hash probe bucket list iters', 'count', 'iterations'),
    MetricMeta('buffer time', 'time', 'ms'),
    MetricMeta('build side size', 'size', 'bytes'),
    MetricMeta('build time', 'time', 'ms'),
    MetricMeta('collect batch time', 'time', 'ms'),
    MetricMeta('concat batch time', 'time', 'ms'),
    MetricMeta('data size', 'size', 'bytes'),
    MetricMeta('duration', 'time', 'ms'),
    MetricMeta('fetch wait time', 'time', 'ms'),
    MetricMeta('internal.metrics.diskBytesSpilled', 'size', 'bytes'),
    MetricMeta('internal.metrics.executorCpuTime', 'time', 'ns'),
    MetricMeta('internal.metrics.executorDeserializeCpuTime', 'time', 'ns'),
    MetricMeta('internal.metrics.executorDeserializeTime', 'time', 'ms'),
    MetricMeta('internal.metrics.executorRunTime', 'time', 'ms'),
    MetricMeta('internal.metrics.input.bytesRead', 'size', 'bytes'),
    MetricMeta('internal.metrics.input.recordsRead', 'count', 'records'),
    MetricMeta('internal.metrics.jvmGCTime', 'time', 'ms'),
    MetricMeta('internal.metrics.memoryBytesSpilled', 'size', 'bytes'),
    MetricMeta('internal.metrics.output.bytesWritten', 'size', 'bytes'),
    MetricMeta('internal.metrics.output.recordsWritten', 'count', 'records'),
    MetricMeta('internal.metrics.peakExecutionMemory', 'size', 'bytes'),
    MetricMeta('internal.metrics.resultSerializationTime', 'time', 'ms'),
    MetricMeta('internal.metrics.resultSize', 'size', 'bytes'),
    MetricMeta('internal.metrics.shuffle.read.fetchWaitTime', 'time', 'ms'),
    MetricMeta('internal.metrics.shuffle.read.localBlocksFetched', 'count', 'blocks'),
    MetricMeta('internal.metrics.shuffle.read.localBytesRead', 'size', 'bytes'),
    MetricMeta('internal.metrics.shuffle.read.recordsRead', 'count', 'records'),
    MetricMeta('internal.metrics.shuffle.read.remoteBlocksFetched', 'count', 'blocks'),
    MetricMeta('internal.metrics.shuffle.read.remoteBytesRead', 'size', 'bytes'),
    MetricMeta('internal.metrics.shuffle.read.remoteBytesReadToDisk', 'size', 'bytes'),
    MetricMeta('internal.metrics.shuffle.write.bytesWritten', 'size', 'bytes'),
    MetricMeta('internal.metrics.shuffle.write.recordsWritten', 'count', 'records'),
    MetricMeta('internal.metrics.shuffle.write.writeTime', 'time', 'ms'),
    MetricMeta('join output rows', 'count', 'rows'),
    MetricMeta('join time', 'time', 'ms'),
    MetricMeta('local blocks read', 'count', 'blocks'),
    MetricMeta('local bytes read', 'size', 'bytes'),
    MetricMeta('number of input columnar batches', 'count', 'batches'),
    MetricMeta('number of input rows', 'count', 'rows'),
    MetricMeta('number of output columnar batches', 'count', 'batches'),
    MetricMeta('number of output rows', 'count', 'rows'),
    MetricMeta('peak device memory', 'size', 'bytes'),
    MetricMeta('peak memory', 'size', 'bytes'),
    MetricMeta('records read', 'count', 'records'),
    MetricMeta('remote blocks read', 'count', 'blocks'),
    MetricMeta('remote bytes read', 'size', 'bytes'),
    MetricMeta('scan time', 'time', 'ms'),
    MetricMeta('shuffle bytes written', 'size', 'bytes'),
    MetricMeta('shuffle records written', 'count', 'records'),
    MetricMeta('shuffle write time', 'time', 'ms'),
    MetricMeta('spill size', 'size', 'bytes'),
    MetricMeta('sort time', 'time', 'ms'),
    MetricMeta('time in aggregation build', 'time', 'ms'),
    MetricMeta('time in batch concat', 'time', 'ms'),
    MetricMeta('time in compute agg', 'time', 'ms'),
    MetricMeta('total time', 'time', 'ns'),
    MetricMeta('write time', 'time', 'ms')
]

In [ ]:
metric_meta = spark.createDataFrame(data=metric_metas)
task_metrics = tidy_tasks(metrics)

In [ ]:
tasks_to_plans = with_appmeta(task_metrics.join(accumulable_nodes, "accumulatorId").join(plan_nodes, "plan_node"))


In [ ]:
tasks_to_plans.groupBy("nodeName").count().toPandas()

In [ ]:
import altair as alt
alt.data_transformers.enable('json')

def stage_and_task_charts(task_metrics_df, noun="Time"):
    
    selection = alt.selection_multi(name="SelectorName", fields=['Stage ID'])
    stage_metrics_df = task_metrics_df.groupby(['Stage ID', 'Metric Name']).sum()
    
    stages = alt.Chart(
        stage_metrics_df.reset_index()
    ).mark_bar().encode(
        x='Stage ID:N',
        y=alt.Y('sum(Metric Value):Q', title=noun),
        color='Metric Name:N',
        tooltip=['Metric Name', 'Metric Value', 'Task ID']
    ).add_selection(selection).interactive()
    
    tasks = alt.Chart(
        task_metrics_df.reset_index()
    ).mark_bar().encode(
        x='Task ID:N',
        y=alt.Y('sum(Metric Value):Q', title=noun),
        color='Metric Name:N',
        tooltip=['Metric Name', 'Metric Value', 'Task ID']
    ).transform_filter(
        selection
    ).interactive()

    return alt.vconcat(stages, tasks)

def layered_stage_and_task_charts(task_layers, noun="Time"):
    
    selection = alt.selection_multi(name="selector_SelectorName", fields=['Stage ID'])
    sdfs = [tdf.groupby(['Stage ID', 'Metric Name']).sum() for tdf in task_layers]
    
    stages = alt.layer(*[alt.Chart(
        sdf.reset_index()
    ).mark_bar().encode(
        x='Stage ID:N',
        y=alt.Y('sum(Metric Value):Q', title=noun),
        color='Metric Name:N',
        tooltip=['Metric Name', 'Metric Value', 'Task ID']
    ) for sdf in sdfs]).add_selection(selection).interactive()
    
    tasks = alt.layer(*[alt.Chart(
        tdf.reset_index()
    ).mark_bar().encode(
        x='Task ID:N',
        y=alt.Y('sum(Metric Value):Q', title=noun),
        color='Metric Name:N',
        tooltip=['Metric Name', 'Metric Value', 'Task ID']
    ).transform_filter(
        selection
    ) for tdf in task_layers]).interactive()

    return alt.vconcat(stages, tasks)


# Identifying available metrics

In [ ]:
task_metrics.select("Metric Name").distinct().orderBy("Metric Name").toPandas()

# Shuffle metrics

In [ ]:
task_byte_metrics = tidy_tasks(metrics).join(
    metric_meta.withColumnRenamed("MetricName", "Metric Name"), 
    "Metric Name", 
    how="outer"
).where(F.col("unit") == "bytes").groupBy("Stage ID", "Task ID", "Metric Name").sum("Metric Value").withColumnRenamed("sum(Metric Value)", "Metric Value").toPandas()


In [ ]:
task_shuffle_metrics = task_byte_metrics[task_byte_metrics['Metric Name'].str.contains('internal.metrics.shuffle')].sort_values('Task ID')
shuffle_replacer = lambda match: "Shuffle %s" % match.group('metric')
task_shuffle_metrics['Metric Name'] = task_shuffle_metrics['Metric Name'].str.replace(r'internal\.metrics\.shuffle\.(?P<kind>read|write).(?P<metric>.*)$', shuffle_replacer)

In [ ]:
stage_and_task_charts(task_shuffle_metrics, "bytes")

# Executor time metrics

In [ ]:
task_metrics = tidy_tasks(metrics).join(
    metric_meta.withColumnRenamed("MetricName", "Metric Name"), 
    "Metric Name", 
    how="outer"
).withColumn("Metric Value", F.col("Metric Value").cast("float"))

task_ms_metrics = task_metrics.where(F.col("unit") == "ms").groupBy("Stage ID", "Task ID", "Metric Name").sum("Metric Value").withColumnRenamed("sum(Metric Value)", "Metric Value")
task_ns_metrics = task_metrics.where(F.col("unit") == "ns").groupBy("Stage ID", "Task ID", "Metric Name").sum("Metric Value").withColumnRenamed("sum(Metric Value)", "Metric Value").withColumn("Metric Value", F.col("Metric Value").cast("float") / 1000000)

task_time_metrics = task_ms_metrics.union(task_ns_metrics).toPandas()

In [ ]:
task_metrics

In [ ]:
task_executor_metrics = task_time_metrics[~task_time_metrics['Metric Name'].str.contains('internal.metrics.shuffle.')].sort_values('Task ID')

stage_and_task_charts(task_executor_metrics)

# Plotting wall-clock vs CPU time with layered charts

This gives us some sense of the relationship between CPU time and system time.

In [ ]:
cputime = task_time_metrics[task_time_metrics['Metric Name'].str.contains('executorCpuTime')]
runtime = task_time_metrics[task_time_metrics['Metric Name'].str.contains('executorRunTime')]
layered_stage_and_task_charts([runtime, cputime])

# Memory and spill metrics

In [ ]:
stage_and_task_charts(task_byte_metrics[task_byte_metrics['Metric Name'].str.contains('memory') | task_byte_metrics['Metric Name'].str.contains('size') | task_byte_metrics['Metric Name'].str.contains('pill')], "bytes")

In [ ]:
stage_and_task_charts(task_byte_metrics, "bytes")


# Configuration information

In [ ]:
def melt(df, id_vars=None, value_vars=None, var_name='variable', value_name='value'):
    if id_vars is None:
        id_vars = []
    
    if value_vars is None:
        value_vars = [c for c in df.columns if c not in id_vars]
    
    return df.withColumn(
        "value_tuple",
        F.explode(
            F.array(
                *[
                    F.struct(
                        F.lit(vv).alias(var_name), 
                        F.col("`%s`" % vv).alias(value_name)
                    ) 
                    for vv in value_vars
                ]
            )
        )
    ).select(*(id_vars + [F.col("value_tuple")[cn].alias(cn) for cn in [var_name, value_name]]))

In [ ]:
def meltconfig(raw_df, event):
    if event is not None:
        if isinstance(event, list):
            df = raw_df.where(F.col("Event").isin(event))
        else:
            df = raw_df.where(F.col("Event") == event)
    else:
        df = raw_df
            
    def helper(df, field):
        return melt(df.select(field).dropna().select("%s.*" % field))

    return helper(df, "Properties").union(helper(df, "System Properties")).union(helper(df, "Hadoop Properties")).distinct()
    

In [ ]:
configs = meltconfig(metrics, ["SparkListenerEnvironmentUpdate","SparkListenerJobStart"])
configs.toPandas()

# Exporting tabular data

In [ ]:
import sqlite3
conn = sqlite3.Connection(output_file)

In [ ]:
with_appmeta(configs).toPandas().to_sql("configs", conn, index=False, if_exists='append')

In [ ]:
task_metrics_table, task_meta_table = split_metrics(task_metrics)

task_metrics_table.toPandas().to_sql('task_metrics', conn, index=False, if_exists='append')
task_meta_table.toPandas().to_sql('task_meta', conn, index=False, if_exists='append')

In [ ]:
task_all = task_metrics_table.join(task_meta_table, ["Task ID", "Stage ID", "Application ID", "Application Name"])
task_all = task_all.toPandas()

In [ ]:
project_columns = ['Application ID', 'Application Name', 'Attempt', 'Executor ID', 'Failed', 'Finish Time', 'Getting Result Time', 'Host', 'Index', 'Killed', 'Launch Time', 'Locality', 'Metric Name', 'Metric Value', 'Speculative', 'Stage ID', 'Task ID']

index_columns = ['Application ID','Application Name','Attempt','Executor ID','Failed','Finish Time','Getting Result Time','Host','Index','Killed','Launch Time','Locality','Speculative','Stage ID','Task ID']

wide_tasks = task_all[
    project_columns
].pivot_table(index=index_columns, columns="Metric Name").reset_index()


wide_tasks.to_sql('wide_tasks', conn, index=False, if_exists='append')



In [ ]:
plan_nodes.toPandas().to_sql('plans', conn, index=False, if_exists='append')
accumulable_nodes.toPandas().to_sql('accumulables', conn, index=False, if_exists='append')

In [ ]:
conn.close()